In [1]:
import numpy as np
import pandas as pd
import re
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import keras
import tensorflow
# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding
from keras.layers import LSTM, Input, TimeDistributed, SimpleRNN
from keras.models import Model
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split, cross_val_score

# Import the backend
from keras import backend as K

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

/Users/c/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/c/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
with open('dna.txt', 'r') as myfile:
  dna = myfile.read()
dna_basepairs = dna
dna_basepairs = re.sub(r'[0-9]([0-9])?([0-9])?([0-9])?([0-9])?([0-9])?', '', dna_basepairs)
dna_basepairs = re.sub(r'\n', '', dna_basepairs)
dna_basepairs = re.sub(r'\s+', '', dna_basepairs).strip()

Create population of arrays that span 50 index values.

In [3]:
def population(data):
    sequence_one = data[:78122]
    sequence_two = data[78123:]
    
    array = np.zeros((len(sequence_one),8))
    vectorize_sequence = {
        'a' : 1,
        't' : 2,
        'g' : 3,
        'c' : 4
    }

    for base in range(len(sequence_one)):
        base_one = sequence_one[base]
        base_two = sequence_two[base]
        array[base, vectorize_sequence[base_one]] = 1
        array[base, vectorize_sequence[base_two] + 3] = 1

    return np.asarray(array)

In [4]:
sequences = population(dna_basepairs)

In [5]:
sequences[:10]

array([[0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0., 0.]])

In [6]:
sequences.shape

(78122, 8)

In [7]:
sequences = sequences.reshape(1, 78122, 8)

looping through previous and current index of the population in order to pass each sequence through the RNN for evaluation

In [8]:
def fitness(train):

    # initialize model
    model = Sequential()
    # add Long short term memory RNN and a dense layer. Compile the model
    model.add(LSTM(8, input_shape=(train.shape[1],8), activation='tanh', return_sequences=True))
    model.add(Dense(8, activation='tanh'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer ='adam')
    print(model.summary())
    # fit the model and fit the model with the previous individual as the train and the current individual as the test
    
    model.fit(train, epochs=3,verbose=0, batch_size = 40);


In [9]:
fitness(sequences)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 78122, 8)          544       
_________________________________________________________________
dense_1 (Dense)              (None, 78122, 8)          72        
Total params: 616
Trainable params: 616
Non-trainable params: 0
_________________________________________________________________
None


IndexError: list index out of range

In [ ]:
def grade()

In [ ]:
def evolve()

In [ ]:
#train, validate, test = np.split(sequences, [int(.5*len(sequences)), int(.75*len(sequences))])
#view the various lengths of the split data.
#print('Train length:', len(train), '\n', 'Test length:', len(test), '\n', 'Validation length:', len(validate))

In [ ]:
# #create the population of individuals to be tested
# def population(data, n, length):
#     sequence_one = []
#     sequence_two = []
#     for i in range(0,n,length):
#     # grab from i to i + 200
#         if i < n / 2:
#             individual = data[i:i+length]
#             sequence_one.append(np.asarray(individual))
#         else:
#             individual = data[i:i+length]
#             sequence_two.append(np.asarray(individual))
#     return [sequence_one + sequence_two]